In [ ]:
# predict next word based on last one

In [ ]:
!pip install -q tensorflow-datasets tensorflow

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
small_dataset = tf.data.Dataset.from_tensor_slices(["primo secondo terzo quarto quinto sesto settimo ottavo nono decimo"])

for element in small_dataset.take(1):
    print(element)

In [ ]:
file_text_content = ""
with open("./notebooks/il-piccolo-principe.txt", "r") as file:
    file_text_content = file.read()
medium_dataset = tf.data.Dataset.from_tensor_slices(file_text_content.split("\n"))

for element in medium_dataset.take(1):
    print(element)

In [ ]:
large_dataset = tfds.load('wikipedia', split='train').map(lambda x: x['text']).take(100)

for element in large_dataset.take(1):
    print(element)

In [ ]:
dataset = small_dataset

In [ ]:
# maybe use subword tokenizer https://www.tensorflow.org/text/guide/subwords_tokenizer

text_vectorizer = tf.keras.layers.TextVectorization(
    output_mode="int",
    standardize="lower_and_strip_punctuation",
)

text_vectorizer.adapt(dataset)

END_OF_SEQUENCE_TOKEN_INDEX = text_vectorizer.vocabulary_size()
text_vectorizer.set_vocabulary(text_vectorizer.get_vocabulary() + ["[END_OF_SEQUENCE]"])
vocabulary_size = text_vectorizer.vocabulary_size() + 1

print(text_vectorizer.get_vocabulary())

In [ ]:
SEQUENCE_WINDOW_SIZE = 2

In [ ]:
def truncate_and_pad_sequence(sequence):
    assert len(sequence.shape) == 1
    sequence = sequence[-SEQUENCE_WINDOW_SIZE:]
    sequence = tf.pad(sequence, [[max(0, SEQUENCE_WINDOW_SIZE - sequence.shape[0]), 0]])
    return sequence

In [ ]:
def generate_training_samples():
    for whole_text_token_indices in dataset.map(text_vectorizer):
        for i in range(len(whole_text_token_indices)):
            yield (
                truncate_and_pad_sequence(whole_text_token_indices[:i]),
                whole_text_token_indices[i],
            )
        yield (
            truncate_and_pad_sequence(whole_text_token_indices),
            END_OF_SEQUENCE_TOKEN_INDEX,
        )


training_dataset = tf.data.Dataset.from_generator(
    generate_training_samples,
    output_signature=(
        tf.TensorSpec(shape=(SEQUENCE_WINDOW_SIZE,), dtype=tf.int64),
        tf.TensorSpec(shape=(), dtype=tf.int64),
    ),
).cache()

for (input, output) in training_dataset.take(10):
    print(([text_vectorizer.get_vocabulary()[token_index] for token_index in input], text_vectorizer.get_vocabulary()[output]))

In [ ]:
input_layer = tf.keras.Input(shape=(SEQUENCE_WINDOW_SIZE), dtype=tf.int64)

word_embedding_layer = tf.keras.layers.Embedding(
    input_dim=vocabulary_size,
    output_dim=32, # smallest common word embedding dimensionality
)

token_selector_layer = tf.keras.layers.Dense(vocabulary_size, activation=tf.nn.softmax)

layer = input_layer
layer = word_embedding_layer(layer)
layer = layer[..., -1, :]
layer = token_selector_layer(layer)

output_layer = layer

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.summary()

history = model.fit(
    training_dataset.prefetch(tf.data.AUTOTUNE).batch(256),
    epochs=20,
)

In [ ]:
def text_completion(input_text, max_inferred_tokens=10):
    input_token_indices = text_vectorizer(input_text)
    output_token_indices = tf.constant([], dtype=tf.int64)
    while len(output_token_indices) < max_inferred_tokens:
        model_input = truncate_and_pad_sequence(
            tf.concat([input_token_indices, output_token_indices], axis=0)
        )
        token_indices_probabilities = model.predict(model_input[tf.newaxis, :])[0]
        next_token_index = tf.argmax(token_indices_probabilities)
        if next_token_index == END_OF_SEQUENCE_TOKEN_INDEX:
            break
        output_token_indices = tf.concat(
            [output_token_indices, [next_token_index]], axis=0
        )
    output_text = " ".join([text_vectorizer.get_vocabulary()[token_index] for token_index in output_token_indices])
    return (input_text, output_text, len(output_token_indices))

In [ ]:
print(text_completion("primo"))
print(text_completion("secondo"))
print(text_completion("terzo"))
print(text_completion("quarto"))
print(text_completion("quinto"))
print(text_completion("sesto"))